In [1]:
import pandas as pd
import glob
import os

import tensorflow as tf
from tensorflow import keras

Read input files as a list.

In [2]:
t = glob.glob("test_files/TABLE_*.tsv") # replace this with your input files

In [3]:
classes = ["saprotroph","necrotroph","hemibiotroph","biotroph"]

Parse input files to a pandas dataframe where the rows correspond to each file and the columns to all annotations.

In [4]:
l=[]
for i in t:
    t = pd.read_csv(i,"\t",usecols=["step_name","step_value"])
    name = i.split("TABLE_")[-1].split(".")[0]
    t.columns = ["name",name]
    t.set_index("name",inplace=True)
    l.append(t)

In [5]:
Xt= pd.concat(l,axis=1)

In [6]:
Xt = Xt.transpose()

In [7]:
Xt.values

array([[1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [8]:
Xt

name,Phospho-2-dehydro-3-deoxyheptonate aldolase,3-dehydroquinate synthase,3-dehydroquinate dehydratase,Shikimate 5-dehydrogenase,Shikimate kinase,3-phosphoshikimate 1-carboxyvinyltransferase,Chorismate synthase,LLM-family F420-associated subfamilies,F420-dependent oxidoreductase families,"Methylene-5,6,7,8-tetrahydromethanopterin dehydrogenase",...,Calcium/calmodulin-dependent protein kinase type II,Insulin release from secretory granules,Glucokinase,TCA cycle,Transient receptor potential cation channel subfamily M member 4,Sodium/potassium-transporting ATPase subunit alpha-1,Sodium/potassium-transporting ATPase subunit beta-1,ATP-binding cassette sub-family C member 8,ATP-sensitive inward rectifier potassium channel 11,Voltage-dependent L-type calcium channel subunits alpha
GCA_000517465,1,1,1,1,1,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
pseudozyma,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Read model.

In [9]:
model = tf.keras.models.load_model('lspred_2_12_20.h5')

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 5204)              27086820  
_________________________________________________________________
dense_1 (Dense)              (None, 224)               1165920   
_________________________________________________________________
dense_2 (Dense)              (None, 64)                14400     
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 260       
Total params: 28,267,400
Trainable params: 28,267,400
Non-trainable params: 0
_________________________________________________________________


Run predict method on the model to get probabilities for all output classes.

In [11]:
pred = model.predict(Xt)

In [12]:
print(classes)
print(pred)

['saprotroph', 'necrotroph', 'hemibiotroph', 'biotroph']
[[0.00992905 0.00214383 0.10016803 0.8877591 ]
 [0.30474845 0.25972545 0.24817495 0.18735124]]


Format probabilities in a dataframe.

In [13]:
p = pd.DataFrame(pred)
p.columns = classes
p.index = Xt.index
p

,saprotroph,necrotroph,hemibiotroph,biotroph
GCA_000517465,0.009929,0.002144,0.100168,0.887759
pseudozyma,0.304748,0.259725,0.248175,0.187351
